# Example: Yield to Maturity Calculation for a Zero
In this example, we will calculate the yield to maturity (YTM) of a Zero given its price, face value, discount rate and time to maturity. The YTM is the internal rate of return (IRR) of the instrument.

> __Motivation:__ Fill me in.


Let's get started!
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 

> __Include:__ The [include(...) function](https://docs.julialang.org/en/v1/base/base/#include) is a convenient way to load Julia code from another file. The `Include.jl` file is a Julia file that loads external packages, and various functions that we will use in the exercise. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Otherwise, packages are downloaded and then loaded.

Setup the environment.

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # this sets up the environment, we'll do this all the time, on everything we do

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We are going to look at a hypothetical 52-week U.S. Treasury Bill. The data for these securities is encoded in [a `NamedTuple` data structure](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple), let's say `bill_data::NamedTuple`:

In [2]:
bill_data = (
    T = "52-Week" |> securityterm,
    n = 2,
    par = 100.0,
    r̄ = 0.04538, # 4.53 and some change annualized discount rate
    c̄ = 0.0,   # zero-coupon
);

Let's build a `zero_coupon_model::MyUSTreasuryZeroCouponBondModel` instance using the `bill_data::NamedTuple` data structure.

In [3]:
zero_coupon_model = let

    # initialize -
    discount_model = DiscreteCompoundingModel();
    zero_coupon_model = nothing; # initalize


    # Build a zero-coupon T-bill model using data from the selected bill
    zero_coupon_model = build(MyUSTreasuryZeroCouponBondModel, (
        par = bill_data.par,
        n = bill_data.n, 
        T = bill_data.T,
        rate =  bill_data.r̄,
    )) |> discount_model;

    zero_coupon_model # return
end;

What's the price of this bill at auction? 

In [4]:
zero_coupon_model.price # price per 100 USD par value

95.62366236777791

### Constants
We set some constant values that we use in the calculations below. Please see the comment next to the constant for its meaning, permissible units, etc.

In [5]:
 t = 182.0; # number of days to maturity

___

## Task 1: Let's compute the Yield at Auction and Afterward
In this task, we show that the yield at auction is the same as the yield to maturity (YTM) which in term is the same as the discount rate used in pricing the zero-coupon instrument. The __annualized investment yield__ of a zero-coupon instrument with $t$ days to maturity (units: days), face (par) value $V_{P}$ (units: USD), and purchase price $V_{B}$ at time $t$ (units: USD) is given by:
$$
\boxed{
\begin{align*}
Y(t) & = \underbrace{\left(\frac{V_{P}}{V_{B,t}} - 1\right)}_{\text{HPR}} \times \underbrace{\left(\frac{1}{T}\right)}_{\text{annualize}}\quad\\
\end{align*}}
$$
where $\text{HPR}$ is the __holding period return__ and the $1/T$ term is an __annualization factor__.

> __Idea:__ The yield to maturity (YTM) of a zero-coupon instrument is equal to the annualized investment rate $\bar{r}$, i.e., the discount rate used in pricing the instrument. After the auction, the yield to maturity is no longer equal to the __original__ investment rate $\bar{r}$. It is either higher or lower, depending on the price of the instrument in the secondary market.

So what do we see?


In [6]:
Y,HPR = let

    # initialize -
    T = t/365.0; # annualized time to maturity
    V_P = zero_coupon_model.par; # face value
    V_B = zero_coupon_model.price; # purchase price
    δ = 0.0; # price change in secondary market

    # compute -
    HPR  = V_P/(V_B + δ) - 1; # holding period return
    Y    = HPR / T;     # annualized yield

    (Y,HPR) # return
end

(0.0917839879616208, 0.045766262490452014)

### Discussion
Let's run a few simulations to see what happens when we start changing the price and the time to maturity of our example treasury bill.
1. Suppose we purchased the Bill at auction, and held the Bill for 6-moths (182 days). We checked the price of the bill in the secondary market, and found that the price as the same as the auction price. What is the yield to maturity (YTM) of the bill now?  (Is it higher, lower or the same as the yield, i.e., $\bar{r}$, at auction?)
2. Suppose we purchased the Bill at auction, and held the Bill for 6-moths (182 days). We checked the price of the bill in the secondary market, and found that the price had __increased__ by 1 USD. What is the yield to maturity (YTM) of the bill now?  (Is it higher, lower or the same as the yield, i.e., $\bar{r}$, at auction?)
3. Suppose we purchased the Bill at auction, and held the Bill for 6-moths (182 days). We checked the price of the bill in the secondary market, and found that the price had __decreased__ by 1 USD. What is the yield to maturity (YTM) of the bill now?  (Is it higher, lower or the same as the yield, i.e., $\bar{r}$, at auction?)

___

## Task 2: How are the yields related to the discount rate used in pricing?
In this task, we will explore how the yields are related to the discount rate used in pricing the zero-coupon instrument. The relationship between the yield and the discount (spot) rate, is given by:
$$
\boxed{
\begin{align*}
\bar{r} & = n\left[\left(1 + Y\;T\right)^{1/(nT)} - 1\right]
\end{align*}}
$$

Let's put some numbers in and see what we see.

In [7]:
r̂ = let

    # initialize -
    T = t/365.0; # annualized time to maturity
    V_P = zero_coupon_model.par; # face value
    V_B = zero_coupon_model.price; # purchase price
    n = zero_coupon_model.n; # compounding periods per year

    # what is the discount rate used in pricing?
    r̄ = n*((1 + Y*T)^(1/(n*T)) - 1);
end

0.09178967220000045

Let's check: Is the estimated discount rates the same as the discount rate used in pricing the instrument at auction?

In [8]:
@assert isapprox(r̂, zero_coupon_model.rate; atol=1e-6)

AssertionError: AssertionError: isapprox(r̂, zero_coupon_model.rate; atol = 1.0e-6)

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.
___